In [1]:
import netCDF4 as nc
from netCDF4 import Dataset
from numpy import array
from numpy import nanmean
import numpy as np
from keras.models import Sequential
from statsmodels.tsa.ar_model import AutoReg
from keras.layers import Dense
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from math import sqrt
import math
from shutil import copyfile
import warnings
warnings.filterwarnings('ignore')
#tf.random.set_seed(7)
#tf.random.set_seed(77)

In [2]:
from shutil import copyfile
copyfile('icecon.APPX.1000x1000.nc', 'icecon.APPX.1000x1000.RMSE.MLP_vs_AR.nc')
nc_f    = './icecon.APPX.1000x1000.RMSE.MLP_vs_AR.nc'
#nc_f    = './icecon.APPX.250x250.1982-2017.nc'
nc_fid  = Dataset(nc_f, 'r+')
time    = nc_fid.variables['record'][:]
sic     = nc_fid.variables['icecon'][:]
sic0    = sic[0,:,:]
RMSE_ML = np.empty(shape=sic0.shape,dtype='float') 
RMSE_AR = np.empty(shape=sic0.shape,dtype='float') 
RMSE_ML_AR = np.empty(shape=sic0.shape,dtype='float') 
n_steps = 12
print(sic0)

[[  0.    0.    0.    0.    0.    0.    0.    0.    0.    0. ]
 [  0.    0.    0.    0.    0.    0.    nan  87.9   nan   nan]
 [  nan   nan   0.    nan  98.9   nan   nan   nan   nan   nan]
 [  nan   nan   nan 100.   99.4  99.1   nan   nan   nan   nan]
 [  nan   nan   nan   nan  99.3  99.6   nan   nan   nan   nan]
 [  nan  99.   97.8  98.2  99.7  98.9  99.5   nan   nan   nan]
 [  nan   nan  56.6   nan  96.9  92.9  25.4   nan   nan   nan]
 [  0.    0.    0.    0.    0.    0.    nan   nan   nan   nan]
 [  0.    0.    0.    0.    0.    0.    nan   nan   nan   0. ]
 [  0.    0.    0.    0.    0.    nan   nan   0.    nan   nan]]


In [3]:
print(nc_fid.dimensions)
#print(nc_fid.variables.keys())
print(sic.shape)
print(time.shape)

{'record': <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'record', size = 444, 'num_pixels': <class 'netCDF4._netCDF4.Dimension'>: name = 'num_pixels', size = 10, 'num_lines': <class 'netCDF4._netCDF4.Dimension'>: name = 'num_lines', size = 10, 'ncl3': <class 'netCDF4._netCDF4.Dimension'>: name = 'ncl3', size = 444}
(444, 10, 10)
(444,)


In [4]:
nmonth  = len(time)
sic_train       =  sic[0:nmonth-12,:,:]
time_train      = time[0:nmonth-12]

In [5]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
        X, y = list(), list()
        for i in range(len(sequence)):
                # find the end of this pattern
                end_ix = i + n_steps
                # check if we are beyond the sequence
                if end_ix > len(sequence)-1:
                        break
                # gather input and output parts of the pattern
                seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
                X.append(seq_x)
                y.append(seq_y)
        return array(X), array(y)

def mlp_sequence_sic(sic, n_steps, dense_num, epoch_num):
        X = list()
        if any(np.isnan(sic)):
            for i in range(n_steps):
                X_new = float("nan")
                X.append(X_new)
                #print(X_new.shape)
            RMSE = float("nan")
        else:
            if min(sic) == max(sic):
                for i in range(n_steps):
                    X_new = min(sic)
                    X.append(X_new)
                    #print(X_new.shape)
                RMSE = float("nan")
            else:     
                nmonth          = len(sic)
                sic_train       = sic[0:nmonth-n_steps]
                X_SIC, Y_SIC    = split_sequence(sic_train,n_steps)
                model = Sequential()
                model.add(Dense(dense_num, activation='relu', input_dim=n_steps))
                model.add(Dense(1))
                model.compile(optimizer='adam', loss='mse')
                model.fit(X_SIC, Y_SIC, epochs=epoch_num, verbose=0)
        
        
                for i in range(n_steps):
                    x_input = sic[nmonth-n_steps*2+i:nmonth-n_steps+i]
                    #print(time[nmonth-n_steps*2+i:nmonth-n_steps+i])
                    x_input = x_input.reshape((1, n_steps))
                    #print(x_input)
                    X_new = model.predict(x_input, verbose=0)
                    #print('sic            at ', i,'is ', sic[nmonth-n_steps+i])
                    #print('sic prediction at ', i,'is ', X_new)
                    #print('')
                    #input('press return to continue')
                    X.append(X_new[0,0])
                RMSE = sqrt(mean_squared_error(sic[nmonth-n_steps:], X))
        return array(X), RMSE
    
def AR_sequence_sic(sic, n_steps, lagnum):
        X = list()
        if any(np.isnan(sic)):
            for i in range(n_steps):
                X_new = float("nan")
                X.append(X_new)
                #print(X_new.shape)
            RMSE = float("nan")
        else:
            if min(sic) == max(sic):
                for i in range(n_steps):
                    X_new = min(sic)
                    X.append(X_new)
                    #print(X_new.shape)
                RMSE = float("nan")
            else:     
                for i in range(n_steps):
                    sic_train = sic[0:nmonth-n_steps+i]
                    model = AutoReg(sic_train, lags=lagnum)
                    model_fit = model.fit()
                    X_new = model_fit.predict(start=len(sic_train), end=len(sic_train), dynamic=False)
                    #print('sic            at ', i,'is ', sic[nmonth-n_steps+i])
                    #print('sic prediction at ', i,'is ', X_new)
                    #print('')
                    #input('press return to continue')
                    X.append(X_new)
                RMSE = sqrt(mean_squared_error(sic[nmonth-n_steps:], X))
        return array(X), RMSE

In [6]:
for ix in range(sic.shape[1]):
    for jy in range(sic.shape[2]):
        y_hat, RMSE_ML[ix,jy] = mlp_sequence_sic(sic[:,ix,jy],12,500,200)
        z_hat, RMSE_AR[ix,jy] = AR_sequence_sic(sic[:,ix,jy],12,24)
        RMSE_ML_AR[ix,jy] = RMSE_ML[ix,jy] - RMSE_AR[ix,jy]
        if (not np.isnan(RMSE_ML[ix,jy])):
            print('ix=',ix,'jy=',jy)
            print('RMSE_ML=',round(RMSE_ML[ix,jy]),'RMSE_AR=',round(RMSE_AR[ix,jy]),'ML-AR=',round(RMSE_ML_AR[ix,jy]))
            print('')

ix= 1 jy= 4
RMSE_ML= 2.0 RMSE_AR= 2.0 ML-AR= 0.0

ix= 1 jy= 5
RMSE_ML= 0.0 RMSE_AR= 0.0 ML-AR= -0.0

ix= 1 jy= 7
RMSE_ML= 9.0 RMSE_AR= 7.0 ML-AR= 3.0

ix= 2 jy= 4
RMSE_ML= 14.0 RMSE_AR= 17.0 ML-AR= -3.0

ix= 3 jy= 3
RMSE_ML= 24.0 RMSE_AR= 24.0 ML-AR= 0.0

ix= 3 jy= 4
RMSE_ML= 6.0 RMSE_AR= 14.0 ML-AR= -8.0

ix= 3 jy= 5
RMSE_ML= 8.0 RMSE_AR= 11.0 ML-AR= -3.0

ix= 4 jy= 4
RMSE_ML= 1.0 RMSE_AR= 1.0 ML-AR= -0.0

ix= 4 jy= 5
RMSE_ML= 2.0 RMSE_AR= 3.0 ML-AR= -0.0

ix= 5 jy= 1
RMSE_ML= 12.0 RMSE_AR= 20.0 ML-AR= -8.0

ix= 5 jy= 2
RMSE_ML= 6.0 RMSE_AR= 12.0 ML-AR= -5.0

ix= 5 jy= 3
RMSE_ML= 10.0 RMSE_AR= 11.0 ML-AR= -0.0

ix= 5 jy= 4
RMSE_ML= 3.0 RMSE_AR= 3.0 ML-AR= 0.0

ix= 5 jy= 5
RMSE_ML= 8.0 RMSE_AR= 9.0 ML-AR= -1.0

ix= 5 jy= 6
RMSE_ML= 13.0 RMSE_AR= 9.0 ML-AR= 5.0

ix= 6 jy= 2
RMSE_ML= 9.0 RMSE_AR= 11.0 ML-AR= -1.0

ix= 6 jy= 4
RMSE_ML= 10.0 RMSE_AR= 10.0 ML-AR= 0.0

ix= 6 jy= 5
RMSE_ML= 12.0 RMSE_AR= 10.0 ML-AR= 2.0

ix= 6 jy= 6
RMSE_ML= 1.0 RMSE_AR= 1.0 ML-AR= 0.0

ix= 7 jy= 1
RMSE_ML= 1

In [7]:
print(RMSE_ML)
print('Average of RMSE with MLP = ', nanmean(RMSE_ML))

[[        nan         nan         nan         nan         nan         nan
          nan         nan         nan         nan]
 [        nan         nan         nan         nan  2.3098196   0.05621579
          nan  9.27766406         nan         nan]
 [        nan         nan         nan         nan 13.85467966         nan
          nan         nan         nan         nan]
 [        nan         nan         nan 23.76186227  6.00552813  8.10100734
          nan         nan         nan         nan]
 [        nan         nan         nan         nan  0.88147117  2.18663258
          nan         nan         nan         nan]
 [        nan 12.08432199  6.08206449 10.38184581  2.73372734  8.01130878
  13.38935951         nan         nan         nan]
 [        nan         nan  9.44939719         nan 10.44205727 12.37907512
   1.46419598         nan         nan         nan]
 [        nan 12.07058745         nan         nan         nan         nan
          nan         nan         nan         nan]


In [8]:
print(RMSE_AR)
print('Average of RMSE with Autoregression = ', nanmean(RMSE_AR))

[[        nan         nan         nan         nan         nan         nan
          nan         nan         nan         nan]
 [        nan         nan         nan         nan  2.24674795  0.06059764
          nan  6.52739625         nan         nan]
 [        nan         nan         nan         nan 16.73468578         nan
          nan         nan         nan         nan]
 [        nan         nan         nan 23.69078131 13.57823142 10.80711994
          nan         nan         nan         nan]
 [        nan         nan         nan         nan  0.92024681  2.58481573
          nan         nan         nan         nan]
 [        nan 20.18383132 11.53602915 10.68889664  2.65882914  8.8159868
   8.72396772         nan         nan         nan]
 [        nan         nan 10.68401352         nan 10.29219889 10.31746725
   1.01875851         nan         nan         nan]
 [        nan  8.71869014         nan         nan         nan         nan
          nan         nan         nan         nan]
 

In [9]:
#nc_fid.createDimension('num_pixels',10)
#nc_fid.createDimension('num_lines', 10)
ARvariable = nc_fid.createVariable('RMSE_AR', 'float',('num_pixels', 'num_lines'))
MLvariable = nc_fid.createVariable('RMSE_ML', 'float',('num_pixels', 'num_lines'))
ML_AR      = nc_fid.createVariable('RMSE_ML_AR', 'float',('num_pixels', 'num_lines'))
ARvariable[:] = RMSE_AR[:,:]
MLvariable[:] = RMSE_ML[:,:]
ML_AR[:]      = RMSE_ML_AR[:,:]
nc_fid.close()

In [10]:
print('Average of RMSE with MLP = ', nanmean(RMSE_ML))
print('Average of RMSE with Autoregression = ', nanmean(RMSE_AR))

Average of RMSE with MLP =  7.860072872868727
Average of RMSE with Autoregression =  8.617910734716483
